### Motion Controls
##### Goal  
Modular Code So blocks for specific functions or buttons
First Identify What functions are necessary for experiment context

#### Features from LabView Code 

Accepts these Inputs:  
IP Address: 192.168.254.254  
Port No: 5001  
Timeout: 10,000  
GroupName: Group 1  
nbitems: 1  
TargetPosition:  
STOP BUTTON  

1. Create xps object and Checks group status below

### Connect and Status

In [1]:
%load_ext autoreload
%autoreload 2   
from newportxps.newportxps import NewportXPS

xps = NewportXPS('192.168.254.254', username='Administrator', password='Administrator')

print(xps.status_report())

# XPS host:         192.168.254.254 (XPS-1871)
# Firmware:         XPS-RL-D-N16002
# Current Time:     Tue May 14 17:15:24 2024
# Last Reboot:      Tue May 14 15:21:13 2024
# Trajectory Group: None
# Groups and Stages
Group1 (singleaxisinuse), Status: Ready state from motion
   Group1.Pos (ILS@ILS200LM@XPS-DRV02)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK


In [4]:
xps.read_systemini()

{'Group1': {'category': 'singleaxisinuse', 'positioners': ['Pos']}}

Error Description:
ZM high level: This refers to the mechanical zero signal used with some stages. The ZM
signal is high during one part of the travel and low during the other part of the travel.
The detection of the ZM high/low transition in combination with an encoder index pulse
signal allows a fast and repeatable origin search
(MechanicalZeroAndIndexHomeSearch).

Do I need to resolve this error? Don't think so has not prevented the running of following commands



In [15]:
# Print all stages configured on the controller
for sname, info in xps.stages.items():
    print(sname, xps.get_stage_position(sname), info)

Group1.Pos 0 {'stagetype': 'ILS@ILS200LM@XPS-DRV02', 'max_velo': 500, 'max_accel': 666.6666666666666, 'low_limit': -100, 'high_limit': 100}


2. Now Initialize Group  
Notes:  
Initializing the group (turning the motor on) causes the stage to move.  
Have not neccessarily observed pattern in how much the stage moves or even the direction it moves.  
Comparing the command above and below to check the position of the group shows the change.
AFter initializing group Status is in Not Referenced State


Note:  To prevent following Error during Homing process. Ensure that within the stages.ini configuration file the Home Search Process Parameters are as below the maximum velocity is 40 Unit/Sec.  
SETTINGS:  
; --- Home search process parameters  
; --- <HomeSearch.MechanicalZeroAndIndexHomeSearch>  
HomeSearchSequenceType = MechanicalZeroAndIndexHomeSearch  
HomeSearchMaximumVelocity = 30 ; Unit/Sec  
HomeSearchMaximumAcceleration = 400 ; Unit/Sec²  
HomeSearchTimeOut = 10 ; Sec  
HomingSensorOffset = 0 ; Unit  

### Initialize and Home

In [2]:
#if group status enabled then kill all groups else initialize group
# Remember to Kill Group before exiting or if the motor is still on
xps.kill_group('Group1')

# Turns on the servo Motor for the specified group and home the stage
xps.initialize_group('Group1', with_encoder=True, home=True, with_raise=True)

In [6]:
# Print all groups configured on the controller
for gname, info in xps.groups.items():
    print(gname, info)

Group1 {'category': 'singleaxisinuse', 'positioners': ['Pos']}


In [7]:
# Note Turning the servo on will cause the stage to move the stage position
# Check to make sure that these functions perform as expected
print('Where Group1.Pos is', xps.get_stage_position('Group1.Pos'), info)
print(xps.get_group_status())

Where Group1.Pos is 1e-05 {'category': 'singleaxisinuse', 'positioners': ['Pos']}
{'Group1': 'Ready state from homing'}


### Move Motion Stage

In [11]:
# Move the stage to a new position. Default is absolute motion can optionally set the motion to relative by relative=True
xps.move_stage('Group1.Pos', 0.0)
xps.move_stage('Group1.Pos', 10.0)
xps.move_stage('Group1.Pos', -10.0)
xps.move_stage('Group1.Pos', 20.0)
xps.move_stage('Group1.Pos', -20.0)
xps.move_stage('Group1.Pos', 30.0)
xps.move_stage('Group1.Pos', -30.0)
xps.move_stage('Group1.Pos', 40.0)
xps.move_stage('Group1.Pos', -40.0)
xps.move_stage('Group1.Pos', 0.0)

''

### STOP Button